# Cat & Dog Classification with Tensorflow

In [ ]:
# Import libaries
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import os 
import cv2
import time

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Activation

In [ ]:
# AUTOTUNE to set the parameters dynamically based on available CPU
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
# Path to train & test data
train_dir = './data/train/'
test_dir = './data/test/'

# All image paths 
train_data_path = [train_dir + file_name for file_name in os.listdir(train_dir)]

## Load and Preprocess Data

In [ ]:
IMAGE_SIZE = 224

# Preprocess an image
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image /= 255.0

    return image

# Read the image from path and preprocess
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [ ]:
# Create dataset from image paths
path_dataset = tf.data.Dataset.from_tensor_slices(train_data_path)

# Create image dataset from path dataset
image_dataset = path_dataset.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

In [ ]:
# Generate image labels
image_label = list(map(lambda x: 1 if x.split('.')[0] == 'cat' else 0, os.listdir(train_dir)))

# Create image label dataset
label_dataset = tf.data.Dataset.from_tensor_slices(tf.cast(image_label, tf.int64))

In [ ]:
# Combine image dataset and image label dataset
dataset = tf.data.Dataset.zip((image_dataset, label_dataset))

In [ ]:
# Split train, test and validation set
DATASET_SIZE = 25000
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 4096

train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

dataset = dataset.shuffle(buffer_size = SHUFFLE_BUFFER_SIZE)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [ ]:
# Perform mini-batch in train_dataset and test_dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# Define model
def define_model():
    # load model
    model = tf.keras.applications.Xception(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = tf.keras.Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = tf.keras.optimizers.RMSprop(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = define_model()

In [ ]:
# Train model
model_history = model.fit(train_dataset, epochs = 50, validation_data = val_dataset)

In [ ]:
# Evaluate model
model.evaluate(test_dataset)

In [ ]:
# Save model
model.save('catdog_classifier_Xception.h5')